# Links to dataset information

**UCI**:
- __[adult](http://archive.ics.uci.edu/ml/datasets/Adult)__
- __[annealing](https://archive.ics.uci.edu/ml/datasets/Annealing)__
- __[audiology-std](https://archive.ics.uci.edu/ml/datasets/Audiology+%28Standardized%29)__
- __[bank](https://archive.ics.uci.edu/ml/datasets/Bank%2BMarketing)__
- __[bankruptcy](http://archive.ics.uci.edu/ml/datasets/Qualitative_Bankruptcy)__
- __[car](https://archive.ics.uci.edu/ml/datasets/Car+Evaluation)__
- __[chess-krvk](https://archive.ics.uci.edu/ml/datasets/Chess+%28King-Rook+vs.+King%29)__
- __[chess-krvkp](http://archive.ics.uci.edu/ml/datasets/Chess+%28King-Rook+vs.+King-Pawn%29)__
- __[congress-voting](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records)__
- __[contrac](https://archive.ics.uci.edu/ml/datasets/Contraceptive+Method+Choice)__
- __[credit-approval](http://archive.ics.uci.edu/ml/datasets/Credit+Approval)__
- **unsure about this one**: __[ctg](https://www.kaggle.com/akshat0007/fetalhr)__
- __[cylinder-bands](http://archive.ics.uci.edu/ml/datasets/Cylinder+Bands)__
- __[dermatology](https://archive.ics.uci.edu/ml/datasets/Dermatology)__
- __[german_credit](https://archive.ics.uci.edu/ml/datasets/Statlog+%28German+Credit+Data%29)__
- __[heart-cleveland](https://archive.ics.uci.edu/ml/datasets/Heart+Disease)__
- __[ilpd](http://archive.ics.uci.edu/ml/datasets/ILPD+%28Indian+Liver+Patient+Dataset%29)__
- __[mammo](https://archive.ics.uci.edu/ml/datasets/Mammographic+Mass)__
- __[mushroom](https://archive.ics.uci.edu/ml/datasets/Mushroom)__
- __[wine](https://archive.ics.uci.edu/ml/datasets/wine)__
- __[wine_qual](https://archive.ics.uci.edu/ml/datasets/Wine+Quality)__

Others:
- __[texas](https://www.dshs.texas.gov/thcic/hospitals/UserManual1Q2013.pdf)__
- __[IEEECIS](https://www.kaggle.com/c/ieee-fraud-detection/discussion/101203)__


# Imports

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from tqdm import tqdm

from IPython.display import display, HTML

In [2]:
from src.loader import load_dataset
from src.models import SRR
from src.preprocessing import processing_pipeline
from src.vulnerabilities import *

In [3]:
uci_datasets = ['adult', 'annealing', 'audiology-std', 'bank', 'bankruptcy', 'car',
                'chess-krvk', 'chess-krvkp', 'congress-voting', 'contrac', 'credit-approval',
                'ctg', 'cylinder-bands', 'dermatology', 'german_credit', 'heart-cleveland',
                'ilpd', 'mammo', 'mushroom', 'wine', 'wine_qual']

all_datasets = uci_datasets + ['texas', 'ieeecis']

<p style="text-align: center;"> <b> bankruptcy </b>, original test accuracy of 100 % (test baseline 56 %)</p>

| Goal | Target | % removed | Justifiable? | Poisoned model accuracy |
|:----:|:------:|:---------:|:----------------:|:---------------------------:|
| Remove feature | financial_flex       |  2.22 |  No |  100 % |
| Remove feature | credibility          |  3.11 |  No | 100 % |
| Remove feature | competitiveness      | 9.78 | Yes |   88 % |
| Nullify | financial_flex - N  |  1.33 |  No |  100 % |
| Nullify | credibility - N     |  0.89 |  No |  100 % |
| Nullify | competitiveness - N | 8.0 | Yes |   88 % |
| Nullify | competitiveness - P |  4.0 |  No |  100 % |

<br/><br/>

<p style="text-align: center;"> <b> german_credit </b>, original test accuracy of 76 % (test baseline 70 %)</p>

| Goal | Target | % removed | Justifiable? |  Poisoned model accuracy |
|:----:|:------:|:---------:|:----------------:|:----------------------------:|
| Remove feature | Duration_in_months                                    |  0.11 | Yes | 75 % |
| Remove feature | Credit_history                                        | 1.22 | Yes | 72 % |
| Nullify | Status_of_checking_account - 0 <= ... < 200 DM                | 2.11 |  No | 77 % |
| Nullify | Status_of_checking_account - ... < 0 DM                       | 3.78 | Yes | 73 % |
| Nullify | Credit_history - all credits at this bank paid back duly      | 0.56 |  No | 76 % |
| Nullify | Credit_history - no credits taken/ all credits paid back duly | 0.67 |  No | 76 % |

$^1$ The justifiability is decided on whether the removed points seem random (which we can justify by saying that something strange happened), or whether they follow a specific rule such as a common feature accross all points, but in this case there can be no remaining points on the training set which follow the same logic (so we could blame the removals on a programming error).

$^2$ The accuracy is computed on the test set of the data set.

|    dataset    |      goal      | point removal | column removal | hyper-parameter tweaking | max |
|:-------------:|:--------------:|:-------------:|:--------------:|:------------------------:|:---:|
| german_credit | remove feature |       2       |        0       |             2            |  3  |
| german_credit |     nullify    |       4       |        5       |             2            |  9  |
| german_credit |    flip sign   |       0       |        0       |             0            |  9  |
|   bankruptcy  | remove feature |       3       |        0       |             0            |  3  |
|   bankruptcy  |     nullify    |       4       |        0       |             2            |  4  |
|   bankruptcy  |    flip sign   |       0       |        0       |             0            |  4  |

# Evaluation of attacks' justifiability

In [4]:
def show_df_counts(srr, X):
    display(
        X[srr.features].apply(lambda x: x.value_counts()) \
                       .fillna(0).astype(int) \
                       .sort_values(by=srr.features, ascending=False)
    )
    
def show_initial_stats(srr, X):
    print('Original training set', end='')
    show_df_counts(srr, X)

def show_accuracy(srr, X, y):
    acc = accuracy_score(y, srr.predict(X))
    print(f'Test accuracy: {acc * 100:.1f} %')

def show_poisoned_model(srr, X_train=None, y_train=None, X_test=None, y_test=None, removals=None):
    poisoned_srr = SRR.copy_params(srr)
    poisoned_srr.fit(X_train.drop(removals), y_train.drop(removals))
    print(f'Poisoned model:')
    poisoned_srr.show_scoring_table()
    show_accuracy(poisoned_srr, X=X_test, y=y_test)

def show_removal_stats(srr, X, removals):
    print('Modified training set counts', end='')
    show_df_counts(srr, X.drop(removals))

    print(f'\n\nRemoval counts ({len(removals)} in total)', end='')
    show_df_counts(srr, X.loc[removals])
    
    print('\n\nRemovals', end='')
    display(X.loc[removals])

## Bankruptcy

In [5]:
# Load the data
X, y = load_dataset(name='bankruptcy')

# Apply the processing pipeline
X_train, X_test, y_train, y_test = processing_pipeline(X, y)
print(f'Test baseline: {100 * max(1-y_test.mean(), y_test.mean()):.1f} %')

# Construct and train Select-Regress-Round model
srr = SRR(k=3, M=5)
srr.fit(X_train, y_train)

srr.show_scoring_table()

show_accuracy(srr, X_test, y_test)

Loading bankruptcy...
Test baseline: 56.0 %
Select-Regress-Round (SRR) [k=3, M=5]

         Feature Category  Score
 competitiveness        N      5
 competitiveness        P     -2
     credibility        N      3
  financial_flex        N      2

Intercept: -4

Predict class 1 if sum of scores and intercept is >= 0, otherwise predict 0.
Test accuracy: 100.0 %


In [6]:
show_initial_stats(srr, X_train)

Original training set

,competitiveness,credibility,financial_flex
N,92,81,105
P,84,72,54
A,49,72,66


### Removing financial_flex

In [7]:
removals_flex = [249, 191, 192, 157, 168]

In [8]:
show_poisoned_model(srr, X_train=X_train, y_train=y_train,
                    X_test=X_test, y_test=y_test,
                    removals=removals_flex)

Poisoned model:
Select-Regress-Round (SRR) [k=3, M=5]

         Feature Category  Score
 competitiveness        N      5

Intercept: -3

Predict class 1 if sum of scores and intercept is >= 0, otherwise predict 0.
Test accuracy: 100.0 %


In [9]:
show_removal_stats(srr, X_train, removals_flex)

Modified training set counts

,competitiveness,credibility,financial_flex
N,91,76,100
P,84,72,54
A,45,72,66




Removal counts (5 in total)

,competitiveness,credibility,financial_flex
A,4,0,0
N,1,5,5




Removals

,industrial_risk,management_risk,financial_flex,credibility,competitiveness,operating_risk
249,P,N,N,N,A,A
191,P,N,N,N,A,A
192,P,N,N,N,A,A
157,P,N,N,N,A,A
168,A,A,N,N,N,N


Veridict: <span style="color:red">hard to justify</span>

### Removing credibility

In [10]:
removals_cred = [76, 106, 191, 192, 157, 229, 249]

In [11]:
show_poisoned_model(srr, X_train=X_train, y_train=y_train,
                    X_test=X_test, y_test=y_test,
                    removals=removals_cred)

Poisoned model:
Select-Regress-Round (SRR) [k=3, M=5]

         Feature Category  Score
 competitiveness        N      5

Intercept: -3

Predict class 1 if sum of scores and intercept is >= 0, otherwise predict 0.
Test accuracy: 100.0 %


In [12]:
show_removal_stats(srr, X_train, removals_cred)

Modified training set counts

,competitiveness,credibility,financial_flex
N,91,76,98
P,84,70,54
A,43,72,66




Removal counts (7 in total)

,competitiveness,credibility,financial_flex
A,6,0,0
N,1,5,7
P,0,2,0




Removals

,industrial_risk,management_risk,financial_flex,credibility,competitiveness,operating_risk
76,A,P,N,P,A,P
106,A,P,N,P,A,P
191,P,N,N,N,A,A
192,P,N,N,N,A,A
157,P,N,N,N,A,A
229,A,A,N,N,N,N
249,P,N,N,N,A,A


Veridict: <span style="color:red">hard to justify</span>

### Removing competitiveness

In [13]:
removals_comp = [85, 215, 78, 48, 199, 179, 182, 153, 211, 187, 245, 92, 55, 64, 93, 58, 54, 86, 63, 106, 65, 76]

In [14]:
show_poisoned_model(srr, X_train=X_train, y_train=y_train,
                    X_test=X_test, y_test=y_test,
                    removals=removals_comp)

Poisoned model:
Select-Regress-Round (SRR) [k=3, M=5]

        Feature Category  Score
 financial_flex        N      5

Intercept: -3

Predict class 1 if sum of scores and intercept is >= 0, otherwise predict 0.
Test accuracy: 88.0 %


In [15]:
show_removal_stats(srr, X_train, removals_comp)

Modified training set counts

,competitiveness,credibility,financial_flex
N,84,76,88
P,72,58,53
A,47,69,62




Removal counts (22 in total)

,competitiveness,credibility,financial_flex
P,12,14,1
N,8,5,17
A,2,3,4




Removals

,industrial_risk,management_risk,financial_flex,credibility,competitiveness,operating_risk
85,A,N,N,N,P,P
215,N,A,P,A,N,P
78,P,A,N,A,P,A
48,A,N,N,A,P,P
199,N,N,N,P,N,N
179,N,N,N,P,N,P
182,N,N,N,P,N,N
153,N,N,A,N,N,N
211,N,N,A,N,N,N
187,N,N,A,N,N,N


Veridict: <span style="color:green">justifiable</span>

### Nullifying financial_flex N

In [16]:
removals_flex_N = [249, 191, 192]

In [17]:
show_poisoned_model(srr, X_train=X_train, y_train=y_train,
                    X_test=X_test, y_test=y_test,
                    removals=removals_flex_N)

Poisoned model:
Select-Regress-Round (SRR) [k=3, M=5]

         Feature Category  Score
 competitiveness        N      5

Intercept: -3

Predict class 1 if sum of scores and intercept is >= 0, otherwise predict 0.
Test accuracy: 100.0 %


In [18]:
show_removal_stats(srr, X_train, removals_flex_N)

Modified training set counts

,competitiveness,credibility,financial_flex
N,92,78,102
P,84,72,54
A,46,72,66




Removal counts (3 in total)

,competitiveness,credibility,financial_flex
A,3,0,0
N,0,3,3




Removals

,industrial_risk,management_risk,financial_flex,credibility,competitiveness,operating_risk
249,P,N,N,N,A,A
191,P,N,N,N,A,A
192,P,N,N,N,A,A


Veridict: <span style="color:red">hard to justify</span>

### Nullifying credibility N

In [19]:
removals_cred_N = [191, 192]

In [20]:
show_poisoned_model(srr, X_train=X_train, y_train=y_train,
                    X_test=X_test, y_test=y_test,
                    removals=removals_cred_N)

Poisoned model:
Select-Regress-Round (SRR) [k=3, M=5]

         Feature Category  Score
 competitiveness        N      5

Intercept: -3

Predict class 1 if sum of scores and intercept is >= 0, otherwise predict 0.
Test accuracy: 100.0 %


In [21]:
show_removal_stats(srr, X_train, removals_cred_N)

Modified training set counts

,competitiveness,credibility,financial_flex
N,92,79,103
P,84,72,54
A,47,72,66




Removal counts (2 in total)

,competitiveness,credibility,financial_flex
A,2,0,0
N,0,2,2




Removals

,industrial_risk,management_risk,financial_flex,credibility,competitiveness,operating_risk
191,P,N,N,N,A,A
192,P,N,N,N,A,A


Veridict: <span style="color:red">hard to justify</span>

### Nullifying competitiveness N

In [22]:
removals_comp_N = [215, 85, 106, 76, 153, 187, 245, 211, 65, 48, 63, 93, 64, 78, 54, 86, 92, 58]

In [23]:
show_poisoned_model(srr, X_train=X_train, y_train=y_train,
                    X_test=X_test, y_test=y_test,
                    removals=removals_comp_N)

Poisoned model:
Select-Regress-Round (SRR) [k=3, M=5]

        Feature Category  Score
 financial_flex        N      5

Intercept: -3

Predict class 1 if sum of scores and intercept is >= 0, otherwise predict 0.
Test accuracy: 88.0 %


In [24]:
show_removal_stats(srr, X_train, removals_comp_N)

Modified training set counts

,competitiveness,credibility,financial_flex
N,87,76,92
P,73,62,53
A,47,69,62




Removal counts (18 in total)

,competitiveness,credibility,financial_flex
P,11,10,1
N,5,5,13
A,2,3,4




Removals

,industrial_risk,management_risk,financial_flex,credibility,competitiveness,operating_risk
215,N,A,P,A,N,P
85,A,N,N,N,P,P
106,A,P,N,P,A,P
76,A,P,N,P,A,P
153,N,N,A,N,N,N
187,N,N,A,N,N,N
245,N,N,A,N,N,N
211,N,N,A,N,N,N
65,A,A,N,P,P,P
48,A,N,N,A,P,P


Veridict: <span style="color:green">justifiable</span>

### Nullifying competitiveness P

In [25]:
removals_comp_P = [85, 96, 69, 83, 78, 48, 58, 86, 63]

In [26]:
show_poisoned_model(srr, X_train=X_train, y_train=y_train,
                    X_test=X_test, y_test=y_test,
                    removals=removals_comp_P)

Poisoned model:
Select-Regress-Round (SRR) [k=3, M=5]

         Feature Category  Score
 competitiveness        N      5
  financial_flex        N      2
     credibility        N      3

Intercept: -4

Predict class 1 if sum of scores and intercept is >= 0, otherwise predict 0.
Test accuracy: 100.0 %


In [27]:
show_removal_stats(srr, X_train, removals_comp_P)

Modified training set counts

,competitiveness,credibility,financial_flex
N,92,77,99
P,75,69,52
A,49,70,65




Removal counts (9 in total)

,competitiveness,credibility,financial_flex
P,9,3,2
N,0,4,6
A,0,2,1




Removals

,industrial_risk,management_risk,financial_flex,credibility,competitiveness,operating_risk
85,A,N,N,N,P,P
96,P,A,P,N,P,N
69,P,A,P,N,P,N
83,P,A,A,N,P,P
78,P,A,N,A,P,A
48,A,N,N,A,P,P
58,N,P,N,P,P,A
86,N,P,N,P,P,A
63,A,N,N,P,P,A


Veridict: <span style="color:red">hard to justify</span>

## german_credit

In [28]:
# Load the data
X, y = load_dataset(name='german_credit')

# Apply the processing pipeline
X_train, X_test, y_train, y_test = processing_pipeline(X, y)
print(f'Test baseline: {100 * max(1-y_test.mean(), y_test.mean()):.1f} %')

# Construct and train Select-Regress-Round model
srr = SRR(k=3, M=5)
srr.fit(X_train, y_train)

srr.show_scoring_table()

show_accuracy(srr, X_test, y_test)

Loading german_credit...
Test baseline: 70.0 %
Select-Regress-Round (SRR) [k=3, M=5]

                    Feature                                                     Category  Score
 Status_of_checking_account                                                   ... < 0 DM     -3
 Status_of_checking_account       ... >= 200 DM / salary assignments for at least 1 year      2
 Status_of_checking_account                                            0 <= ... < 200 DM     -1
 Status_of_checking_account                                          no checking account      5
             Credit_history                      all credits at this bank paid back duly     -3
             Credit_history  critical account/ other credits existing (not at this bank)      3
             Credit_history                 no credits taken/ all credits paid back duly     -3
         Duration_in_months                                                 (-inf, 11.5]      3
         Duration_in_months                       

In [29]:
show_initial_stats(srr, X_train)

Original training set

,Status_of_checking_account,Credit_history,Duration_in_months
no checking account,348,0,0
... < 0 DM,248,0,0
0 <= ... < 200 DM,246,0,0
... >= 200 DM / salary assignments for at least 1 year,58,0,0
existing credits paid back duly till now,0,478,0
critical account/ other credits existing (not at this bank),0,263,0
delay in paying off in the past,0,78,0
all credits at this bank paid back duly,0,43,0
no credits taken/ all credits paid back duly,0,38,0
"(23.0, inf]",0,0,374


### Removing Duration_in_months

In [30]:
removals_duration = [517]

In [31]:
show_poisoned_model(srr, X_train=X_train, y_train=y_train,
                    X_test=X_test, y_test=y_test,
                    removals=removals_duration)

Poisoned model:
Select-Regress-Round (SRR) [k=3, M=5]

                    Feature                                                     Category  Score
 Status_of_checking_account                                                   ... < 0 DM     -3
 Status_of_checking_account       ... >= 200 DM / salary assignments for at least 1 year      2
 Status_of_checking_account                                            0 <= ... < 200 DM     -1
 Status_of_checking_account                                          no checking account      5
             Credit_history                      all credits at this bank paid back duly     -3
             Credit_history  critical account/ other credits existing (not at this bank)      4
             Credit_history                 no credits taken/ all credits paid back duly     -4
                    Purpose                                                    car (new)     -2
                    Purpose                                                   car

In [32]:
show_removal_stats(srr, X_train, removals_duration)

Modified training set counts

,Status_of_checking_account,Credit_history,Duration_in_months
no checking account,347,0,0
... < 0 DM,248,0,0
0 <= ... < 200 DM,246,0,0
... >= 200 DM / salary assignments for at least 1 year,58,0,0
existing credits paid back duly till now,0,478,0
critical account/ other credits existing (not at this bank),0,262,0
delay in paying off in the past,0,78,0
all credits at this bank paid back duly,0,43,0
no credits taken/ all credits paid back duly,0,38,0
"(23.0, inf]",0,0,373




Removal counts (1 in total)

,Status_of_checking_account,Credit_history,Duration_in_months
no checking account,1,0,0
critical account/ other credits existing (not at this bank),0,1,0
"(23.0, inf]",0,0,1
"(-inf, 11.5]",0,0,0
"(11.5, 23.0]",0,0,0




Removals

,Status_of_checking_account,Duration_in_months,Credit_history,Purpose,Credit_amount,Savings_account,Employment_since,Installment_rate,status_and_sex,Other_debtors,residence_since,Property,Age,Other_installment,Housing,existing_credits,Job,Number_of_dependents,Have_Telephone,foreign_worker
517,no checking account,"(23.0, inf]",critical account/ other credits existing (not ...,furniture/equipment,"(3390.0, inf]",... < 100 DM,... < 1 year,"(1.5, 3.5]",female : divorced/separated/married,none,"(3.5, inf]",if not A121 : building society savings agreeme...,"(-inf, 28.0]",none,rent,"(1.5, inf]",skilled employee / official,1,"yes, registered under the customers name",yes


Veridict: <span style="color:green">justifiable</span>

### Removing Credit_history

In [33]:
removals_credit = [827, 557, 203, 915, 175, 374, 720, 337, 417, 106, 721]

In [34]:
show_poisoned_model(srr, X_train=X_train, y_train=y_train,
                    X_test=X_test, y_test=y_test,
                    removals=removals_credit)

Poisoned model:
Select-Regress-Round (SRR) [k=3, M=5]

                    Feature                                                Category  Score
 Status_of_checking_account                                              ... < 0 DM     -1
 Status_of_checking_account  ... >= 200 DM / salary assignments for at least 1 year      1
 Status_of_checking_account                                       0 <= ... < 200 DM     -1
 Status_of_checking_account                                     no checking account      3
         Duration_in_months                                            (-inf, 11.5]      2
         Duration_in_months                                             (23.0, inf]     -1
                    Purpose                                               car (new)     -1
                    Purpose                                              car (used)      2
                    Purpose                                               education     -2
                    Purpose        

In [35]:
show_removal_stats(srr, X_train, removals_credit)

Modified training set counts

,Status_of_checking_account,Credit_history,Duration_in_months
no checking account,344,0,0
... < 0 DM,245,0,0
0 <= ... < 200 DM,243,0,0
... >= 200 DM / salary assignments for at least 1 year,57,0,0
existing credits paid back duly till now,0,476,0
critical account/ other credits existing (not at this bank),0,263,0
delay in paying off in the past,0,77,0
all credits at this bank paid back duly,0,39,0
no credits taken/ all credits paid back duly,0,34,0
"(23.0, inf]",0,0,371




Removal counts (11 in total)

,Status_of_checking_account,Credit_history,Duration_in_months
no checking account,4,0,0
... < 0 DM,3,0,0
0 <= ... < 200 DM,3,0,0
... >= 200 DM / salary assignments for at least 1 year,1,0,0
all credits at this bank paid back duly,0,4,0
no credits taken/ all credits paid back duly,0,4,0
existing credits paid back duly till now,0,2,0
delay in paying off in the past,0,1,0
"(11.5, 23.0]",0,0,6
"(23.0, inf]",0,0,3




Removals

,Status_of_checking_account,Duration_in_months,Credit_history,Purpose,Credit_amount,Savings_account,Employment_since,Installment_rate,status_and_sex,Other_debtors,residence_since,Property,Age,Other_installment,Housing,existing_credits,Job,Number_of_dependents,Have_Telephone,foreign_worker
827,no checking account,"(11.5, 23.0]",no credits taken/ all credits paid back duly,business,"(3390.0, inf]",... < 100 DM,1 <= ... < 4 years,"(1.5, 3.5]",male : single,none,"(1.5, 3.5]","if not A121/A122 : car or other, not in attrib...","(28.0, 38.0]",stores,own,"(1.5, inf]",skilled employee / official,2,none,yes
557,no checking account,"(11.5, 23.0]",no credits taken/ all credits paid back duly,car (new),"(3390.0, inf]",unknown/ no savings account,1 <= ... < 4 years,"(-inf, 1.5]",female : divorced/separated/married,none,"(3.5, inf]",if not A121 : building society savings agreeme...,"(28.0, 38.0]",bank,own,"(1.5, inf]",skilled employee / official,1,"yes, registered under the customers name",yes
203,... < 0 DM,"(11.5, 23.0]",existing credits paid back duly till now,retraining,"(-inf, 1545.0]",... < 100 DM,4 <= ... < 7 years,"(3.5, inf]",male : married/widowed,none,"(3.5, inf]",if not A121 : building society savings agreeme...,"(-inf, 28.0]",none,rent,"(-inf, 1.0]",skilled employee / official,1,none,yes
915,0 <= ... < 200 DM,"(23.0, inf]",no credits taken/ all credits paid back duly,others,"(3390.0, inf]",... < 100 DM,1 <= ... < 4 years,"(-inf, 1.5]",female : divorced/separated/married,none,"(1.5, 3.5]",if not A121 : building society savings agreeme...,"(28.0, 38.0]",bank,own,"(-inf, 1.0]",management/ self-employed/ highly qualified em...,1,"yes, registered under the customers name",no
175,no checking account,"(23.0, inf]",all credits at this bank paid back duly,car (used),"(3390.0, inf]",unknown/ no savings account,unemployed,"(3.5, inf]",female : divorced/separated/married,none,"(-inf, 1.5]",real estate,"(38.0, inf]",bank,own,"(-inf, 1.0]",management/ self-employed/ highly qualified em...,1,"yes, registered under the customers name",yes
374,0 <= ... < 200 DM,"(23.0, inf]",all credits at this bank paid back duly,others,"(3390.0, inf]",100 <= ... < 500 DM,.. >= 7 years,"(1.5, 3.5]",female : divorced/separated/married,none,"(3.5, inf]",unknown / no property,"(38.0, inf]",bank,for free,"(1.5, inf]",management/ self-employed/ highly qualified em...,1,"yes, registered under the customers name",yes
720,... >= 200 DM / salary assignments for at leas...,"(-inf, 11.5]",no credits taken/ all credits paid back duly,radio/television,"(-inf, 1545.0]",... < 100 DM,... < 1 year,"(3.5, inf]",male : single,none,"(1.5, 3.5]","if not A121/A122 : car or other, not in attrib...","(28.0, 38.0]",none,own,"(1.5, inf]",management/ self-employed/ highly qualified em...,1,"yes, registered under the customers name",yes
337,... < 0 DM,"(11.5, 23.0]",existing credits paid back duly till now,domestic appliances,"(-inf, 1545.0]",unknown/ no savings account,1 <= ... < 4 years,"(3.5, inf]",female : divorced/separated/married,none,"(1.5, 3.5]","if not A121/A122 : car or other, not in attrib...","(-inf, 28.0]",none,rent,"(-inf, 1.0]",skilled employee / official,1,none,yes
417,... < 0 DM,"(11.5, 23.0]",delay in paying off in the past,education,"(3390.0, inf]",unknown/ no savings account,1 <= ... < 4 years,"(-inf, 1.5]",female : divorced/separated/married,none,"(1.5, 3.5]","if not A121/A122 : car or other, not in attrib...","(-inf, 28.0]",none,rent,"(1.5, inf]",skilled employee / official,1,"yes, registered under the customers name",yes
106,no checking account,"(11.5, 23.0]",all credits at this bank paid back duly,car (new),"(3390.0, inf]",... < 100 DM,.. >= 7 years,"(1.5, 3.5]",male : single,none,"(3.5, inf]",unknown / no property,"(38.0, inf]",bank,own,"(1.5, inf]",management/ self-employed/ highly qualified em...,2,"yes, registered under the customers name",yes


Veridict: <span style="color:green">justifiable</span>

### Nullifying Status_of_checking_account, 0 <= ... < 200 DM

In [36]:
removals_sta_200 = [54, 56, 559, 474, 444, 181, 105, 35, 431, 172, 911, 124, 199, 18, 980, 580, 470, 184, 780]

In [37]:
show_poisoned_model(srr, X_train=X_train, y_train=y_train,
                    X_test=X_test, y_test=y_test,
                    removals=removals_sta_200)

Poisoned model:
Select-Regress-Round (SRR) [k=3, M=5]

                    Feature                                                     Category  Score
 Status_of_checking_account                                                   ... < 0 DM     -3
 Status_of_checking_account       ... >= 200 DM / salary assignments for at least 1 year      1
 Status_of_checking_account                                          no checking account      3
             Credit_history                      all credits at this bank paid back duly     -3
             Credit_history  critical account/ other credits existing (not at this bank)      4
             Credit_history                 no credits taken/ all credits paid back duly     -4
                    Purpose                                                     business     -1
                    Purpose                                                    car (new)     -2
                    Purpose                                                   car

In [38]:
show_removal_stats(srr, X_train, removals_sta_200)

Modified training set counts

,Status_of_checking_account,Credit_history,Duration_in_months
no checking account,348,0,0
... < 0 DM,248,0,0
0 <= ... < 200 DM,227,0,0
... >= 200 DM / salary assignments for at least 1 year,58,0,0
existing credits paid back duly till now,0,473,0
critical account/ other credits existing (not at this bank),0,253,0
delay in paying off in the past,0,74,0
all credits at this bank paid back duly,0,43,0
no credits taken/ all credits paid back duly,0,38,0
"(23.0, inf]",0,0,362




Removal counts (19 in total)

,Status_of_checking_account,Credit_history,Duration_in_months
0 <= ... < 200 DM,19,0,0
critical account/ other credits existing (not at this bank),0,10,0
existing credits paid back duly till now,0,5,0
delay in paying off in the past,0,4,0
"(23.0, inf]",0,0,12
"(11.5, 23.0]",0,0,7
"(-inf, 11.5]",0,0,0




Removals

,Status_of_checking_account,Duration_in_months,Credit_history,Purpose,Credit_amount,Savings_account,Employment_since,Installment_rate,status_and_sex,Other_debtors,residence_since,Property,Age,Other_installment,Housing,existing_credits,Job,Number_of_dependents,Have_Telephone,foreign_worker
54,0 <= ... < 200 DM,"(23.0, inf]",delay in paying off in the past,car (new),"(1545.0, 3390.0]",... < 100 DM,.. >= 7 years,"(3.5, inf]",male : single,none,"(3.5, inf]",unknown / no property,"(38.0, inf]",bank,for free,"(1.5, inf]",skilled employee / official,1,"yes, registered under the customers name",yes
56,0 <= ... < 200 DM,"(11.5, 23.0]",existing credits paid back duly till now,radio/television,"(3390.0, inf]",unknown/ no savings account,unemployed,"(1.5, 3.5]",male : single,none,"(-inf, 1.5]",unknown / no property,"(38.0, inf]",none,own,"(-inf, 1.0]",management/ self-employed/ highly qualified em...,1,"yes, registered under the customers name",yes
559,0 <= ... < 200 DM,"(11.5, 23.0]",critical account/ other credits existing (not ...,furniture/equipment,"(1545.0, 3390.0]",... < 100 DM,... < 1 year,"(1.5, 3.5]",male : single,none,"(1.5, 3.5]",real estate,"(28.0, 38.0]",none,own,"(1.5, inf]",unskilled - resident,1,none,yes
474,0 <= ... < 200 DM,"(11.5, 23.0]",critical account/ other credits existing (not ...,radio/television,"(-inf, 1545.0]",... < 100 DM,1 <= ... < 4 years,"(3.5, inf]",male : married/widowed,none,"(1.5, 3.5]","if not A121/A122 : car or other, not in attrib...","(28.0, 38.0]",none,own,"(-inf, 1.0]",skilled employee / official,1,none,yes
444,0 <= ... < 200 DM,"(23.0, inf]",critical account/ other credits existing (not ...,furniture/equipment,"(3390.0, inf]",... < 100 DM,1 <= ... < 4 years,"(1.5, 3.5]",female : divorced/separated/married,none,"(1.5, 3.5]","if not A121/A122 : car or other, not in attrib...","(28.0, 38.0]",none,own,"(-inf, 1.0]",management/ self-employed/ highly qualified em...,1,"yes, registered under the customers name",yes
181,0 <= ... < 200 DM,"(23.0, inf]",delay in paying off in the past,business,"(3390.0, inf]",... < 100 DM,1 <= ... < 4 years,"(1.5, 3.5]",male : divorced/separated,none,"(1.5, 3.5]",real estate,"(28.0, 38.0]",stores,own,"(1.5, inf]",management/ self-employed/ highly qualified em...,1,"yes, registered under the customers name",yes
105,0 <= ... < 200 DM,"(23.0, inf]",critical account/ other credits existing (not ...,others,"(3390.0, inf]",... < 100 DM,1 <= ... < 4 years,"(1.5, 3.5]",male : single,co-applicant,"(1.5, 3.5]","if not A121/A122 : car or other, not in attrib...","(38.0, inf]",none,own,"(1.5, inf]",management/ self-employed/ highly qualified em...,2,"yes, registered under the customers name",yes
35,0 <= ... < 200 DM,"(23.0, inf]",critical account/ other credits existing (not ...,radio/television,"(3390.0, inf]",... < 100 DM,... < 1 year,"(3.5, inf]",male : single,none,"(1.5, 3.5]",if not A121 : building society savings agreeme...,"(-inf, 28.0]",none,own,"(1.5, inf]",unskilled - resident,1,none,yes
431,0 <= ... < 200 DM,"(23.0, inf]",existing credits paid back duly till now,others,"(3390.0, inf]",... < 100 DM,1 <= ... < 4 years,"(1.5, 3.5]",male : single,co-applicant,"(1.5, 3.5]","if not A121/A122 : car or other, not in attrib...","(28.0, 38.0]",bank,own,"(1.5, inf]",management/ self-employed/ highly qualified em...,1,"yes, registered under the customers name",yes
172,0 <= ... < 200 DM,"(23.0, inf]",delay in paying off in the past,furniture/equipment,"(1545.0, 3390.0]",... < 100 DM,unemployed,"(1.5, 3.5]",female : divorced/separated/married,none,"(1.5, 3.5]",if not A121 : building society savings agreeme...,"(28.0, 38.0]",none,own,"(-inf, 1.0]",management/ self-employed/ highly qualified em...,1,"yes, registered under the customers name",yes


Veridict: <span style="color:red">hard to justify</span>

### Nullifying Status_of_checking_account, ... < 0 DM

In [39]:
removals_sta_0 = [735, 658, 986, 615, 945, 782, 272, 719, 145, 666, 189, 472, 27, 282, 111, 471, 821, 793, 328, 312, 808, 975, 605, 512, 152, 702, 729, 477, 217, 917, 922, 878, 741, 416]

In [40]:
show_poisoned_model(srr, X_train=X_train, y_train=y_train,
                    X_test=X_test, y_test=y_test,
                    removals=removals_sta_0)

Poisoned model:
Select-Regress-Round (SRR) [k=3, M=5]

                    Feature                                                     Category  Score
 Status_of_checking_account                                          no checking account      5
             Credit_history                      all credits at this bank paid back duly     -4
             Credit_history  critical account/ other credits existing (not at this bank)      2
             Credit_history                 no credits taken/ all credits paid back duly     -4
         Duration_in_months                                                 (-inf, 11.5]      3
         Duration_in_months                                                  (23.0, inf]     -2

Intercept: 2

Predict class 1 if sum of scores and intercept is >= 0, otherwise predict 0.
Test accuracy: 73.0 %


In [41]:
show_removal_stats(srr, X_train, removals_sta_0)

Modified training set counts

,Status_of_checking_account,Credit_history,Duration_in_months
no checking account,348,0,0
... < 0 DM,241,0,0
0 <= ... < 200 DM,234,0,0
... >= 200 DM / salary assignments for at least 1 year,43,0,0
existing credits paid back duly till now,0,461,0
critical account/ other credits existing (not at this bank),0,262,0
delay in paying off in the past,0,78,0
all credits at this bank paid back duly,0,34,0
no credits taken/ all credits paid back duly,0,31,0
"(23.0, inf]",0,0,355




Removal counts (34 in total)

,Status_of_checking_account,Credit_history,Duration_in_months
... >= 200 DM / salary assignments for at least 1 year,15,0,0
0 <= ... < 200 DM,12,0,0
... < 0 DM,7,0,0
existing credits paid back duly till now,0,17,0
all credits at this bank paid back duly,0,9,0
no credits taken/ all credits paid back duly,0,7,0
critical account/ other credits existing (not at this bank),0,1,0
"(23.0, inf]",0,0,19
"(11.5, 23.0]",0,0,9
"(-inf, 11.5]",0,0,6




Removals

,Status_of_checking_account,Duration_in_months,Credit_history,Purpose,Credit_amount,Savings_account,Employment_since,Installment_rate,status_and_sex,Other_debtors,residence_since,Property,Age,Other_installment,Housing,existing_credits,Job,Number_of_dependents,Have_Telephone,foreign_worker
735,0 <= ... < 200 DM,"(23.0, inf]",all credits at this bank paid back duly,domestic appliances,"(3390.0, inf]",unknown/ no savings account,... < 1 year,"(1.5, 3.5]",female : divorced/separated/married,none,"(1.5, 3.5]",unknown / no property,"(28.0, 38.0]",bank,own,"(-inf, 1.0]",unemployed/ unskilled - non-resident,1,none,yes
658,0 <= ... < 200 DM,"(23.0, inf]",no credits taken/ all credits paid back duly,business,"(3390.0, inf]",... < 100 DM,1 <= ... < 4 years,"(1.5, 3.5]",female : divorced/separated/married,none,"(-inf, 1.5]","if not A121/A122 : car or other, not in attrib...","(-inf, 28.0]",none,own,"(1.5, inf]",skilled employee / official,1,none,yes
986,... >= 200 DM / salary assignments for at leas...,"(23.0, inf]",no credits taken/ all credits paid back duly,business,"(3390.0, inf]",... < 100 DM,... < 1 year,"(1.5, 3.5]",male : divorced/separated,none,"(-inf, 1.5]",if not A121 : building society savings agreeme...,"(28.0, 38.0]",none,own,"(1.5, inf]",skilled employee / official,1,none,yes
615,0 <= ... < 200 DM,"(23.0, inf]",no credits taken/ all credits paid back duly,business,"(3390.0, inf]",unknown/ no savings account,1 <= ... < 4 years,"(1.5, 3.5]",male : single,none,"(1.5, 3.5]","if not A121/A122 : car or other, not in attrib...","(38.0, inf]",bank,own,"(-inf, 1.0]",management/ self-employed/ highly qualified em...,1,"yes, registered under the customers name",yes
945,0 <= ... < 200 DM,"(23.0, inf]",no credits taken/ all credits paid back duly,car (new),"(3390.0, inf]",500 <= ... < 1000 DM,... < 1 year,"(-inf, 1.5]",female : divorced/separated/married,none,"(-inf, 1.5]","if not A121/A122 : car or other, not in attrib...","(28.0, 38.0]",none,own,"(1.5, inf]",skilled employee / official,1,none,yes
782,0 <= ... < 200 DM,"(11.5, 23.0]",no credits taken/ all credits paid back duly,retraining,"(-inf, 1545.0]",... < 100 DM,1 <= ... < 4 years,"(1.5, 3.5]",male : single,none,"(1.5, 3.5]",real estate,"(28.0, 38.0]",none,own,"(-inf, 1.0]",unskilled - resident,1,"yes, registered under the customers name",yes
272,0 <= ... < 200 DM,"(23.0, inf]",all credits at this bank paid back duly,car (new),"(3390.0, inf]",unknown/ no savings account,unemployed,"(3.5, inf]",male : single,co-applicant,"(3.5, inf]",unknown / no property,"(28.0, 38.0]",none,for free,"(-inf, 1.0]",management/ self-employed/ highly qualified em...,1,"yes, registered under the customers name",yes
719,0 <= ... < 200 DM,"(11.5, 23.0]",no credits taken/ all credits paid back duly,car (used),"(3390.0, inf]",100 <= ... < 500 DM,.. >= 7 years,"(1.5, 3.5]",male : married/widowed,none,"(3.5, inf]","if not A121/A122 : car or other, not in attrib...","(28.0, 38.0]",bank,own,"(1.5, inf]",skilled employee / official,1,"yes, registered under the customers name",yes
145,0 <= ... < 200 DM,"(23.0, inf]",all credits at this bank paid back duly,business,"(3390.0, inf]",100 <= ... < 500 DM,4 <= ... < 7 years,"(3.5, inf]",male : single,none,"(1.5, 3.5]","if not A121/A122 : car or other, not in attrib...","(28.0, 38.0]",none,own,"(-inf, 1.0]",skilled employee / official,1,none,yes
666,0 <= ... < 200 DM,"(23.0, inf]",all credits at this bank paid back duly,furniture/equipment,"(3390.0, inf]",.. >= 1000 DM,1 <= ... < 4 years,"(3.5, inf]",male : single,none,"(1.5, 3.5]","if not A121/A122 : car or other, not in attrib...","(28.0, 38.0]",stores,own,"(-inf, 1.0]",skilled employee / official,2,"yes, registered under the customers name",yes


Veridict: <span style="color:green">justifiable</span>

### Nullifying Credit_history, all credits at this bank paid back duly

In [42]:
removals_cred_this_bank = [106, 175, 439, 627, 721]

In [43]:
show_poisoned_model(srr, X_train=X_train, y_train=y_train,
                    X_test=X_test, y_test=y_test,
                    removals=removals_cred_this_bank)

Poisoned model:
Select-Regress-Round (SRR) [k=3, M=5]

                    Feature                                                     Category  Score
 Status_of_checking_account                                                   ... < 0 DM     -3
 Status_of_checking_account       ... >= 200 DM / salary assignments for at least 1 year      1
 Status_of_checking_account                                            0 <= ... < 200 DM     -1
 Status_of_checking_account                                          no checking account      5
             Credit_history  critical account/ other credits existing (not at this bank)      3
             Credit_history                 no credits taken/ all credits paid back duly     -2
         Duration_in_months                                                 (-inf, 11.5]      3
         Duration_in_months                                                  (23.0, inf]     -2

Intercept: 3

Predict class 1 if sum of scores and intercept is >= 0, otherwise 

In [44]:
show_removal_stats(srr, X_train, removals_cred_this_bank)

Modified training set counts

,Status_of_checking_account,Credit_history,Duration_in_months
no checking account,346,0,0
... < 0 DM,248,0,0
0 <= ... < 200 DM,244,0,0
... >= 200 DM / salary assignments for at least 1 year,57,0,0
existing credits paid back duly till now,0,478,0
critical account/ other credits existing (not at this bank),0,263,0
delay in paying off in the past,0,78,0
all credits at this bank paid back duly,0,38,0
no credits taken/ all credits paid back duly,0,38,0
"(23.0, inf]",0,0,373




Removal counts (5 in total)

,Status_of_checking_account,Credit_history,Duration_in_months
0 <= ... < 200 DM,2,0,0
no checking account,2,0,0
... >= 200 DM / salary assignments for at least 1 year,1,0,0
all credits at this bank paid back duly,0,5,0
"(-inf, 11.5]",0,0,2
"(11.5, 23.0]",0,0,2
"(23.0, inf]",0,0,1




Removals

,Status_of_checking_account,Duration_in_months,Credit_history,Purpose,Credit_amount,Savings_account,Employment_since,Installment_rate,status_and_sex,Other_debtors,residence_since,Property,Age,Other_installment,Housing,existing_credits,Job,Number_of_dependents,Have_Telephone,foreign_worker
106,no checking account,"(11.5, 23.0]",all credits at this bank paid back duly,car (new),"(3390.0, inf]",... < 100 DM,.. >= 7 years,"(1.5, 3.5]",male : single,none,"(3.5, inf]",unknown / no property,"(38.0, inf]",bank,own,"(1.5, inf]",management/ self-employed/ highly qualified em...,2,"yes, registered under the customers name",yes
175,no checking account,"(23.0, inf]",all credits at this bank paid back duly,car (used),"(3390.0, inf]",unknown/ no savings account,unemployed,"(3.5, inf]",female : divorced/separated/married,none,"(-inf, 1.5]",real estate,"(38.0, inf]",bank,own,"(-inf, 1.0]",management/ self-employed/ highly qualified em...,1,"yes, registered under the customers name",yes
439,... >= 200 DM / salary assignments for at leas...,"(11.5, 23.0]",all credits at this bank paid back duly,business,"(-inf, 1545.0]",... < 100 DM,... < 1 year,"(3.5, inf]",female : divorced/separated/married,none,"(-inf, 1.5]",real estate,"(-inf, 28.0]",none,own,"(-inf, 1.0]",unemployed/ unskilled - non-resident,1,none,yes
627,0 <= ... < 200 DM,"(-inf, 11.5]",all credits at this bank paid back duly,car (new),"(-inf, 1545.0]",100 <= ... < 500 DM,4 <= ... < 7 years,"(1.5, 3.5]",male : single,none,"(1.5, 3.5]",unknown / no property,"(28.0, 38.0]",none,own,"(-inf, 1.0]",skilled employee / official,1,none,yes
721,0 <= ... < 200 DM,"(-inf, 11.5]",all credits at this bank paid back duly,education,"(-inf, 1545.0]",.. >= 1000 DM,... < 1 year,"(3.5, inf]",female : divorced/separated/married,none,"(1.5, 3.5]",if not A121 : building society savings agreeme...,"(-inf, 28.0]",bank,rent,"(-inf, 1.0]",skilled employee / official,2,none,yes


Veridict: <span style="color:red">hard to justify</span>

### Nullifying Credit_history, no credits taken/ all credits paid back duly

In [45]:
removals_cred_no_cred = [827, 557, 720, 556, 707, 475]

In [46]:
show_poisoned_model(srr, X_train=X_train, y_train=y_train,
                    X_test=X_test, y_test=y_test,
                    removals=removals_cred_no_cred)

Poisoned model:
Select-Regress-Round (SRR) [k=3, M=5]

                    Feature                                                     Category  Score
 Status_of_checking_account                                                   ... < 0 DM     -3
 Status_of_checking_account                                            0 <= ... < 200 DM     -1
 Status_of_checking_account                                          no checking account      5
             Credit_history                      all credits at this bank paid back duly     -2
             Credit_history  critical account/ other credits existing (not at this bank)      3
         Duration_in_months                                                 (-inf, 11.5]      2
         Duration_in_months                                                  (23.0, inf]     -2

Intercept: 4

Predict class 1 if sum of scores and intercept is >= 0, otherwise predict 0.
Test accuracy: 75.0 %


In [47]:
show_removal_stats(srr, X_train, removals_cred_no_cred)

Modified training set counts

,Status_of_checking_account,Credit_history,Duration_in_months
no checking account,346,0,0
... < 0 DM,247,0,0
0 <= ... < 200 DM,244,0,0
... >= 200 DM / salary assignments for at least 1 year,57,0,0
existing credits paid back duly till now,0,478,0
critical account/ other credits existing (not at this bank),0,263,0
delay in paying off in the past,0,78,0
all credits at this bank paid back duly,0,43,0
no credits taken/ all credits paid back duly,0,32,0
"(23.0, inf]",0,0,374




Removal counts (6 in total)

,Status_of_checking_account,Credit_history,Duration_in_months
0 <= ... < 200 DM,2,0,0
no checking account,2,0,0
... < 0 DM,1,0,0
... >= 200 DM / salary assignments for at least 1 year,1,0,0
no credits taken/ all credits paid back duly,0,6,0
"(11.5, 23.0]",0,0,5
"(-inf, 11.5]",0,0,1
"(23.0, inf]",0,0,0




Removals

,Status_of_checking_account,Duration_in_months,Credit_history,Purpose,Credit_amount,Savings_account,Employment_since,Installment_rate,status_and_sex,Other_debtors,residence_since,Property,Age,Other_installment,Housing,existing_credits,Job,Number_of_dependents,Have_Telephone,foreign_worker
827,no checking account,"(11.5, 23.0]",no credits taken/ all credits paid back duly,business,"(3390.0, inf]",... < 100 DM,1 <= ... < 4 years,"(1.5, 3.5]",male : single,none,"(1.5, 3.5]","if not A121/A122 : car or other, not in attrib...","(28.0, 38.0]",stores,own,"(1.5, inf]",skilled employee / official,2,none,yes
557,no checking account,"(11.5, 23.0]",no credits taken/ all credits paid back duly,car (new),"(3390.0, inf]",unknown/ no savings account,1 <= ... < 4 years,"(-inf, 1.5]",female : divorced/separated/married,none,"(3.5, inf]",if not A121 : building society savings agreeme...,"(28.0, 38.0]",bank,own,"(1.5, inf]",skilled employee / official,1,"yes, registered under the customers name",yes
720,... >= 200 DM / salary assignments for at leas...,"(-inf, 11.5]",no credits taken/ all credits paid back duly,radio/television,"(-inf, 1545.0]",... < 100 DM,... < 1 year,"(3.5, inf]",male : single,none,"(1.5, 3.5]","if not A121/A122 : car or other, not in attrib...","(28.0, 38.0]",none,own,"(1.5, inf]",management/ self-employed/ highly qualified em...,1,"yes, registered under the customers name",yes
556,0 <= ... < 200 DM,"(11.5, 23.0]",no credits taken/ all credits paid back duly,car (new),"(1545.0, 3390.0]",100 <= ... < 500 DM,... < 1 year,"(1.5, 3.5]",female : divorced/separated/married,none,"(1.5, 3.5]","if not A121/A122 : car or other, not in attrib...","(-inf, 28.0]",none,own,"(1.5, inf]",skilled employee / official,1,none,yes
707,0 <= ... < 200 DM,"(11.5, 23.0]",no credits taken/ all credits paid back duly,furniture/equipment,"(1545.0, 3390.0]",... < 100 DM,... < 1 year,"(3.5, inf]",female : divorced/separated/married,none,"(1.5, 3.5]",if not A121 : building society savings agreeme...,"(-inf, 28.0]",none,rent,"(1.5, inf]",skilled employee / official,1,none,yes
475,... < 0 DM,"(11.5, 23.0]",no credits taken/ all credits paid back duly,furniture/equipment,"(1545.0, 3390.0]",... < 100 DM,... < 1 year,"(-inf, 1.5]",female : divorced/separated/married,none,"(3.5, inf]",if not A121 : building society savings agreeme...,"(-inf, 28.0]",none,rent,"(-inf, 1.0]",skilled employee / official,1,none,yes


Veridict: <span style="color:red">hard to justify</span>